3.2 Preparação dos Dados em Formato Tidy

3.2.1 Organização dos Dados

Transformar o dataset limpo para formato tidy data seguindo os princípios fundamentais: cada
variável forma uma coluna, cada observação forma uma linha e cada tipo de unidade observacio-
nal forma uma tabela.

In [1]:
import pandas as pd

In [11]:
# nosso dataset ja é tidy, e isso é observável no nosso csv gerado na etapa anterior: cada diferente variável é uma coluna, cada
# registro forma uma linha e todos eles são exsicatas de uma única tabela.
# então nessa seção o que faremos é reduzir o tamanho do nosso acervo - pois ele ainda contém todos os registros que não mais
# consideraremos, pois eles foram desqualificados em alguma das buscas que fizemos (barcode, estado, país, taxonomista)
# e portanto não serão utilizados.
# PS: registros_biodiversidade.csv está copiado para dentro de etapa_3, caso tenha modificado alguma pesquisa por favor sobrescreva
# o arquivo daqui com o seu!

df = pd.read_csv("registros_biodiversidade.csv", encoding="utf-8-sig", dtype=str)

df_filtrado = df[
    df["barcode_att"].notna() &
    df["stateprovince_att"].notna() &
    df["country_att"].notna() &
    df["identifiedby_att"].notna()
]

df_filtrado.to_csv("registros_biodiversidade_filtrado.csv", index=False, encoding="utf-8-sig", quoting=1)

print(f"dataset tidy reduzido salvo com {len(df)} linhas e {len(df.columns)} colunas.")

dataset tidy reduzido salvo com 185617 linhas e 34 colunas.
